In [35]:
filename = "data/housing.csv"
data = []
# with open(filename, 'r',encoding="utf8") as file:
f = open(filename,'r',encoding="utf8")
df = f.readlines()
for line in df:
    line1=line.split(',')
    data.append(line1)
header = data[0]
values = data[1:]


In [36]:
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql, drop_table_name=None):
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()
    print(rows)
    return rows

def insert_into_tables(conn, sql_insert_statement, values):
    sql = """ INSERT INTO EMPLOYEES VALUES(?, ?, ?) """
    cur = conn.cursor()
    cur.executemany(sql_insert_statement, values)
    return cur.lastrowid

conn = create_connection("database/USRent.db")

In [39]:
housing_table = "CREATE TABLE HousingList (listing_id INTEGER NOT NULL PRIMARY KEY REFERENCES MetaData(listing_id) , price INTEGER, sqfeet INTEGER, beds INTEGER, baths INTEGER, cats_allowed INTEGER, dogs_allowed INTEGER, smoking_allowed INTEGER, wheelchair_access INTEGER, electric_vehicle_charge INTEGER, comes_furnished INTEGER, location_id TEXT NOT NULL REFERENCES Location(location_id), laundry_id TEXT NOT NULL REFERENCES Laundry(laundry_id),housing_type_id TEXT NOT NULL REFERENCES HousingType(housing_type_id), parking_id TEXT NOT NULL REFERENCES Parking(parking_id))"

metadata_table = "CREATE TABLE MetaData (listing_id INTEGER NOT NULL PRIMARY KEY, url TEXT, image_url TEXT)"

location_table = "CREATE TABLE Location (location_id INTEGER NOT NULL PRIMARY KEY, region TEXT, state TEXT)"

laundry_table = "CREATE TABLE Laundry (laundry_id INTEGER NOT NULL PRIMARY KEY, option_name TEXT)"

housingtype_table = "CREATE TABLE HousingType (housing_type_id INTEGER NOT NULL PRIMARY KEY, housing_type TEXT)"

parking_table = "CREATE TABLE Parking (parking_id INTEGER NOT NULL PRIMARY KEY, parking_type TEXT)"

with conn:
    create_table(conn, housing_table, "HousingList")
    create_table(conn, metadata_table, "MetaData")
    create_table(conn, location_table, "Location")
    create_table(conn, laundry_table, "Laundry")
    create_table(conn, housingtype_table, "HousingType")
    create_table(conn, parking_table, "Parking")
options = []

for i in values:
    try:
        options.append(i[15])
    except IndexError:
        continue
options = list(set(options))
insert_laundry_table = "Insert into Laundry(option_name) Values(?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_laundry_table,[(option,) for option in options])

sql_statement = "select * from Laundry;"
execute_sql_statement(sql_statement, conn)


[(1, 'NA'),
 (2, 'laundry in bldg'),
 (3, 'w/d in unit'),
 (4, 'w/d hookups'),
 (5, 'no laundry on site'),
 (6, 'laundry on site')]

In [40]:
type = []

for i in values:
    try:
        type.append(i[5])
    except IndexError:
        continue
type = list(set(type))
insert_type_table = "Insert into HousingType(housing_type) Values(?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_type_table,[(j,) for j in type])

sql_statement = "select * from HousingType;"
execute_sql_statement(sql_statement, conn)


[(1, 'in-law'),
 (2, 'assisted living'),
 (3, 'loft'),
 (4, 'land'),
 (5, 'duplex'),
 (6, 'condo'),
 (7, 'townhouse'),
 (8, 'manufactured'),
 (9, 'cottage/cabin'),
 (10, 'apartment'),
 (11, 'flat'),
 (12, 'house')]

In [41]:
parking = []

for i in values:

    try:
        parking.append(i[16])
    except IndexError:
        continue
parking = list(set(parking))
insert_parking_table = "Insert into Parking(parking_type) Values(?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_parking_table,[(p,) for p in parking])


sql_statement = "select * from Parking;"
execute_sql_statement(sql_statement, conn)


[(1, 'NA'),
 (2, 'valet parking'),
 (3, 'street parking'),
 (4, 'no parking'),
 (5, 'off-street parking'),
 (6, 'detached garage'),
 (7, 'carport'),
 (8, 'attached garage')]

In [42]:
location = []

for i in values:
    try:
        a = i[20].strip()
        location.append(i[2]+'*'+a)
    except IndexError:
        continue

location = list(set(location))

st=[]
loc=[]
for i in location:
    a=i.split('*')
    st.append(a[1])
    loc.append(a[0])

insert_location_table = "Insert into Location(region,state) Values(?,?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_location_table,[(l,s) for l,s in zip(loc,st)])

#have to split and save region and state

sql_statement = "select * from Location;"
execute_sql_statement(sql_statement, conn)

[(1, 'erie', 'pa'),
 (2, 'redding', 'ca'),
 (3, 'olympic peninsula', 'wa'),
 (4, 'bowling green', 'ky'),
 (5, 'victoria', 'tx'),
 (6, 'sacramento', 'ca'),
 (7, 'owensboro', 'ky'),
 (8, 'central michigan', 'mi'),
 (9, 'holland', 'mi'),
 (10, 'wausau', 'wi'),
 (11, 'SF bay area', 'ca'),
 (12, 'southeast missouri', 'mo'),
 (13, 'lehigh valley', 'pa'),
 (14, 'hickory / lenoir', 'nc'),
 (15, 'new orleans', 'la'),
 (16, 'east idaho', 'id'),
 (17, 'fort dodge', 'ia'),
 (18, 'sheboygan', 'wi'),
 (19, 'reading', 'pa'),
 (20, 'york', 'pa'),
 (21, 'yakima', 'wa'),
 (22, 'columbus', 'oh'),
 (23, 'kansas city', 'ks'),
 (24, 'potsdam-canton-massena', 'ny'),
 (25, 'hilton head', 'sc'),
 (26, 'palm springs', 'ca'),
 (27, 'albuquerque', 'nm'),
 (28, 'scranton / wilkes-barre', 'pa'),
 (29, 'cedar rapids', 'ia'),
 (30, 'savannah / hinesville', 'ga'),
 (31, 'lafayette / west lafayette', 'in'),
 (32, 'northern panhandle', 'oh'),
 (33, 'tyler / east TX', 'tx'),
 (34, 'meadville', 'pa'),
 (35, 'del rio / eag

In [48]:
url = []
img_url=[]
l_id = []
for i in values:
    try:
        url.append(i[1])
        img_url.append(i[17])
        l_id.append(i[0])
    except IndexError:
        continue
        
insert_metadata_table = "Insert into MetaData(listing_id,url,image_url) Values(?,?,?);"

with conn:
    cur = conn.cursor()
    cur.executemany(insert_metadata_table,[(i,j,k) for i,j,k in zip(l_id,url,img_url)])

sql_statement = "select * from MetaData;"
execute_sql_statement(sql_statement, conn)

IntegrityError: UNIQUE constraint failed: MetaData.listing_id

In [44]:
def get_type(name,db_name):
    sql_statement = f"select * from {db_name}"
    with conn:
        try:
            c = execute_sql_statement(sql_statement, conn)
            laundry_dict = {x[1]: x[0] for x in c}
            return laundry_dict[name]
        except KeyError:
            return None


In [45]:
def get_location(name):
    sql_statement = "select location_id, region||state from Location"
    with conn:
        try:
            c = execute_sql_statement(sql_statement, conn)
            laundry_dict = {x[1]: x[0] for x in c}
            return laundry_dict[name]
        except KeyError:
            return None

In [47]:
for i in values[0:10]:
    print(i[15],get_type(i[15],"Laundry"))
    print(i[16], get_type(i[16],"Parking"))
    print(i[5], get_type(i[5],"HousingType"))
    location = f"{i[2]}{i[20]}".replace('\n',"")
    print(i[2], get_location(location))
    print("**********************************")

w/d in unit 3
carport 7
apartment 10
reno / tahoe 414
**********************************
w/d hookups 4
carport 7
condo 6
reno / tahoe 414
**********************************
w/d in unit 3
attached garage 8
apartment 10
reno / tahoe 414
**********************************
w/d in unit 3
carport 7
apartment 10
reno / tahoe 414
**********************************
laundry on site 6
NA 1
apartment 10
reno / tahoe 414
**********************************
laundry in bldg 2
NA 1
apartment 10
reno / tahoe 414
**********************************
laundry on site 6
off-street parking 5
apartment 10
reno / tahoe 414
**********************************
w/d in unit 3
carport 7
apartment 10
reno / tahoe 414
**********************************
w/d in unit 3
carport 7
apartment 10
reno / tahoe 414
**********************************
w/d in unit 3
carport 7
condo 6
reno / tahoe 414
**********************************


In [49]:
# housing_table = "CREATE TABLE HousingList (listing_id INTEGER NOT NULL PRIMARY KEY REFERENCES MetaData(listing_id) , price INTEGER, sqfeet INTEGER, beds INTEGER, baths INTEGER, cats_allowed INTEGER, dogs_allowed INTEGER, smoking_allowed INTEGER, wheelchair_access INTEGER, electric_vehicle_charge INTEGER, comes_furnished INTEGER, location_id TEXT NOT NULL REFERENCES Location(location_id), laundry_id TEXT NOT NULL REFERENCES Laundry(laundry_id),housing_type_id TEXT NOT NULL REFERENCES HousingType(housing_type_id), parking_id TEXT NOT NULL REFERENCES Parking(parking_id))"
input_tup = []
for i in values:
    try:
        l_id=i[0]
        price=i[4]
        sq_feet=i[6] 
        bed=i[7]
        bath=i[8]
        cats=i[9]
        dogs=i[10]
        smoking=i[11]
        wchair=i[12]
        e_charge=i[13]
        furnished=i[14]
        location = f"{i[2]}{i[20]}".replace('\n',"")
        loc = get_location(location)
        h_type = get_type(i[5],"HousingType")
        park_type = get_type(i[16],"Parking")
        ldry = get_type(i[15],"Laundry")
        insert_HousingList_table = "Insert into HousingList(listing_id,price,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access, electric_vehicle_charge, comes_furnished, location_id, laundry_id,housing_type_id, parking_id) Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?);"
        input_tup.append((l_id,price,sq_feet,bed,bath,cats,dogs,smoking,wchair,e_charge,furnished,loc,ldry,h_type,park_type))

    except Error as e:
        print(e)
        continue
        
# insert_metadata_table = "Insert into MetaData(listing_id,url,image_url) Values(?,?,?);"

# with conn:
#     cur = conn.cursor()
#     cur.executemany(insert_metadata_table,[(i,j,k) for i,j,k in zip(l_id,url,img_url)])
with conn:
    cur = conn.cursor()
    cur.executemany(insert_HousingList_table,input_tup)

sql_statement = "select count(*) from HousingList;"
execute_sql_statement(sql_statement, conn)

[(7003808130, 799, 954, 2, 2, 1, 1, 1, 0, 0, 0, '213', '4', '10', '1'),
 (7004010416, 1005, 1117, 2, 2, 1, 1, 1, 0, 0, 0, '87', '1', '10', '5'),
 (7004032234, 750, 763, 2, 2, 1, 1, 1, 0, 0, 0, '390', '6', '10', '7'),
 (7004041631, 1000, 800, 1, 0, 0, 1, 1, 0, 0, 0, '77', '2', '10', '7'),
 (7004048100, 530, 641, 1, 1, 1, 1, 1, 0, 0, 0, '128', '1', '10', '1'),
 (7004059925, 850, 1500, 3, 1, 0, 0, 0, 0, 0, 0, '351', '3', '12', '7'),
 (7004071335, 1000, 900, 1, 1, 0, 1, 1, 0, 0, 0, '77', '2', '10', '7'),
 (7004183951, 1500, 1200, 3, 1.5, 1, 0, 1, 0, 0, 0, '373', '3', '6', '3'),
 (7004199151, 545, 440, 0, 1, 1, 1, 1, 0, 0, 0, '390', '6', '10', '7'),
 (7004389140, 1550, 1700, 3, 2.5, 0, 0, 1, 0, 0, 0, '172', '4', '12', '8'),
 (7004529237, 570, 815, 2, 2, 1, 1, 1, 0, 0, 0, '160', '1', '12', '1'),
 (7004546974, 550, 900, 3, 3, 1, 1, 1, 0, 0, 0, '240', '1', '12', '1'),
 (7004554442, 600, 635, 1, 1, 0, 0, 1, 0, 0, 0, '351', '1', '10', '1'),
 (7004558383, 690, 858, 2, 1, 0, 0, 1, 0, 0, 0, '351', 

In [50]:
sql_statement = "select count(*) from HousingList;"
execute_sql_statement(sql_statement, conn)

[(384977,)]